In [39]:
import pandas as pd
import json 
from tqdm import tqdm


def to_df(json_file):
    dataset = json.load(open(json_file))
    title, context, question, id, is_impossible, answer, plausible_answers = [], [], [], [], [], [], []
    for topic in dataset['data']:
        for qas in topic['paragraphs']:
            for pair in qas['qas']:
                title.append(topic['title'])
                context.append(qas['context'])
                question.append(pair['question'])
                id.append(pair['id'])
                try:
                    is_impossible.append(pair['is_impossible'])
                except:
                    is_impossible.append(False)
                if not is_impossible[-1]: 
                    answer.append(pair['answers'])
                    plausible_answers.append([])
                else: 
                    answer.append([])
                    try:
                        plausible_answers.append(pair['plausible_answers'])
                    except:
                        plausible_answers.append([])  
    return pd.DataFrame(zip(title, context, question, id, answer, plausible_answers, is_impossible), columns=['title', 'context', 'question', 'id', 'answer', 'plausible_answers', 'is_impossible'])
                

In [40]:
def convert_to_json(df):
    json_file = {}
    json_file['version'] = 'viquad2_training_set'
    json_file['data'] = []
    for id in tqdm(range(len(df))):
        data = df.loc[id]
        question = {}
        question['question'] = data['question']
        if data['is_impossible']:
            question['answers'] = []
            question['plausible_answers'] = data['plausible_answers']
        else:
            question['answers'] = data['answer']
            question['plausible_answers'] = []
        question['id'] = data['id']
        question['is_impossible'] = bool(data['is_impossible'])
        if len(json_file['data']) == 0:
            topic = {}
            topic['title'] = data['title']
            topic['paragraphs'] = [{'qas':[question], 'context':data['context']}]
            json_file['data'].append(topic)
        else:
            if json_file['data'][-1]['title'] == data['title']:
                    if json_file['data'][-1]['paragraphs'][-1]['context'] == data['context']:
                        json_file['data'][-1]['paragraphs'][-1]['qas'].append(question)
                    else:
                        json_file['data'][-1]['paragraphs'].append({'qas':[question], 'context':data['context']})
            else:
                topic = {}
                topic['title'] = data['title']
                topic['paragraphs'] = [{'qas':[question], 'context':data['context']}]
                json_file['data'].append(topic)
    return json_file


In [41]:
train_df = to_df('/home/brownyeyes/WorkDirectory/Multidocs-MRC/MRC_VLSP/v2_train_ViQuAD.json')
val_df = to_df('/home/brownyeyes/WorkDirectory/Multidocs-MRC/MRC_VLSP/v1_dev_ViQuAD.json')
test_df = to_df('/home/brownyeyes/WorkDirectory/Multidocs-MRC/MRC_VLSP/v1_test_ViQuAD.json')

In [42]:
train_df_new = to_df('/home/brownyeyes/WorkDirectory/Multidocs-MRC/MRC_VLSP/v2_train_ViQuAD_new.json')

In [43]:
val_df

,title,context,question,id,answer,plausible_answers,is_impossible
0,Paris,Paris nằm ở điểm gặp nhau của các hành trình t...,Vị trí địa lý của Paris có gì đặc biệt?,uit_01__00256_0_1,"[{'answer_start': 6, 'text': 'nằm ở điểm gặp n...",[],False
1,Paris,Paris nằm ở điểm gặp nhau của các hành trình t...,Kinh tế xung quanh kinh đô ánh sáng mạnh về gì?,uit_01__00256_0_2,"[{'answer_start': 109, 'text': 'nông nghiệp'},...",[],False
2,Paris,Paris nằm ở điểm gặp nhau của các hành trình t...,Cuộc cách mạng Pháp diễn ra ở đâu?,uit_01__00256_0_3,"[{'answer_start': 358, 'text': 'Paris'}, {'ans...",[],False
3,Paris,Paris nằm ở điểm gặp nhau của các hành trình t...,Kinh đô ánh sáng nổi tiếng về lĩnh vực gì?,uit_01__00256_0_4,"[{'answer_start': 634, 'text': 'nghệ thuật và ...",[],False
4,Paris,Paris nằm ở điểm gặp nhau của các hành trình t...,Paris là một thành phố quan trọng của nước Phá...,uit_01__00256_0_5,"[{'answer_start': 134, 'text': 'thế kỷ 10'}, {...",[],False
...,...,...,...,...,...,...,...
2280,Uganda,"Năm 2010, tổng sản phẩm quốc nội của Uganda đạ...","Con số 17,12 tỷ USD biểu thị thông số kinh tế ...",uit_01__00971_6_1,"[{'answer_start': 10, 'text': 'tổng sản phẩm q...",[],False
2281,Uganda,"Năm 2010, tổng sản phẩm quốc nội của Uganda đạ...","Vào năm 2010, tổng thu nhập cá nhân của mỗi ng...",uit_01__00971_6_2,"[{'answer_start': 153, 'text': '500 USD'}, {'a...",[],False
2282,Uganda,"Năm 2010, tổng sản phẩm quốc nội của Uganda đạ...",Phần lớn nguồn nhân lực phát triển ở khu vực I...,uit_01__00971_6_3,"[{'answer_start': 177, 'text': 'Nông nghiệp'},...",[],False
2283,Uganda,"Năm 2010, tổng sản phẩm quốc nội của Uganda đạ...",Những nông sản chủ lực của nền nông nghiệp tại...,uit_01__00971_6_4,"[{'answer_start': 318, 'text': 'cà phê, chè, n...",[],False


In [44]:
test_df

,title,context,question,id,answer,plausible_answers,is_impossible
0,Đức,"Năm 1871, Đức trở thành một quốc gia dân tộc k...",Cộng hòa Weimar chính thức thay thế đế quốc Đứ...,uit_01__00061_0_1,"[{'answer_start': 124, 'text': 'Chiến tranh th...",[],False
1,Đức,"Năm 1871, Đức trở thành một quốc gia dân tộc k...",Nước Đức hiện nay sự hợp thành của hai nước nà...,uit_01__00061_0_2,"[{'answer_start': 432, 'text': 'Cộng hòa Liên ...",[],False
2,Đức,"Năm 1871, Đức trở thành một quốc gia dân tộc k...",Hậu quả nào đã xảy ra khi tại Đức chế độ độc t...,uit_01__00061_0_3,"[{'answer_start': 311, 'text': 'Chiến tranh th...",[],False
3,Đức,"Năm 1871, Đức trở thành một quốc gia dân tộc k...",Cộng hòa Liên bang Đức và Cộng hòa Dân chủ Đức...,uit_01__00061_0_4,"[{'answer_start': 480, 'text': 'Năm 1990'}, {'...",[],False
4,Đức,"Trong thế kỷ XXI, Đức là một đại cường quốc và...","Vào năm 2015, cả hai lĩnh vực nào Đức đều xếp ...",uit_01__00061_1_1,"[{'answer_start': 223, 'text': 'xuất khẩu và n...",[],False
...,...,...,...,...,...,...,...
2205,Wikipedia,Các yêu cầu trang được gửi cho tầng máy Squid ...,Nếu những yêu cầu mà bộ nhớ Squid không thể th...,uit_01__00059_13_1,"[{'answer_start': 102, 'text': 'được gửi qua c...",[],False
2206,Wikipedia,Các yêu cầu trang được gửi cho tầng máy Squid ...,Cần làm gì để tăng tốc độ trả lời cho người ch...,uit_01__00059_13_2,"[{'answer_start': 431, 'text': 'các trang được...",[],False
2207,Wikipedia,Các yêu cầu trang được gửi cho tầng máy Squid ...,Các máy chủ cân bằng tải gửi yêu cầu cho máy c...,uit_01__00059_13_3,"[{'answer_start': 251, 'text': 'kết xuất trang...",[],False
2208,Wikipedia,Các yêu cầu trang được gửi cho tầng máy Squid ...,Để giải quyết các yêu cầu thì máy chủ web có n...,uit_01__00059_13_4,"[{'answer_start': 304, 'text': 'gửi lại những ...",[],False


In [45]:
negative_value = train_df[train_df['is_impossible']==True].sample(2000)

In [46]:
negative_value

,title,context,question,id,answer,plausible_answers,is_impossible
14573,Cộng hòa Síp,Tháng 1 năm 1950 Nhà thờ Chính thống phương Đô...,Mục đích thành lập của tổ chức Kháng chiến Thổ...,uit_014574,[],"[{'answer_start': 416, 'text': 'tìm kiếm độc l...",True
8346,Malaysia,"Trong thập niên 1970, nền kinh tế dựa chủ yếu ...",Sự bất bình đẳng giữa các dân tộc trong nền cô...,uit_008347,[],"[{'answer_start': 500, 'text': 'người Hoa chiế...",True
3019,Động vật lưỡng cư,Động vật lưỡng cư là các loài động vật có xươn...,Hầu hết động vật cổ đại có hộp sọ như thế nào?,uit_003020,[],"[{'answer_start': 633, 'text': 'rộng và ngắn, ...",True
2891,Thổ Nhĩ Kỳ,"Đến năm 2011, Thổ Nhĩ Kỳ có 166 đại học. Nhập ...",Các học sinh trên thế giới được đưa vào trường...,uit_002892,[],"[{'answer_start': 85, 'text': 'kết quả khảo th...",True
13478,Mặt trận Dân tộc Giải phóng miền Nam Việt Nam,Tuy nhiên sự thật các cải cách của ông kéo the...,Các chính sách cải cách của Ngô Đình Diệm bắt ...,uit_013479,[],"[{'answer_start': 618, 'text': '1959'}]",True
...,...,...,...,...,...,...,...
8141,Khánh Hòa,"Trước thời nhà Nguyễn, vì là đất mới được mở m...",Cấp bậc học nào ngày nay tương đương với bậc Đ...,uit_008142,[],"[{'answer_start': 882, 'text': 'bậc Phổ thông ...",True
1191,Albert Einstein,Các bài báo ban đầu của Einstein bắt nguồn từ ...,Nhà khoa học Einstein đã có chứng cứ về sự tồn...,uit_001192,[],"[{'answer_start': 327, 'text': 'Antoine Lavois...",True
14572,Cộng hòa Síp,Tháng 1 năm 1950 Nhà thờ Chính thống phương Đô...,Cộng đồng người Síp Thổ Nhĩ Kỳ phản ứng thế nà...,uit_014573,[],"[{'answer_start': 117, 'text': 'tẩy chay'}]",True
3930,Roma,Roma đã phát triển thành nơi lưu trữ nghệ thuậ...,"Năm 2010, sự kiện kiến trúc nào đã diễn ra tại...",uit_003931,[],"[{'answer_start': 311, 'text': 'buổi khai trươ...",True


In [47]:
train_df.drop(negative_value.index, inplace=True)
train_df = train_df.reset_index(drop=True)

In [48]:
train_df

,title,context,question,id,answer,plausible_answers,is_impossible
0,Phạm Văn Đồng,Phạm Văn Đồng (1 tháng 3 năm 1906 - 29 tháng 4...,Tên gọi nào được Phạm Văn Đồng sử dụng khi làm...,uit_000001,"[{'answer_start': 507, 'text': 'Lâm Bá Kiệt'}]",[],False
1,Phạm Văn Đồng,Phạm Văn Đồng (1 tháng 3 năm 1906 - 29 tháng 4...,Phạm Văn Đồng giữ chức vụ gì trong bộ máy Nhà ...,uit_000002,"[{'answer_start': 60, 'text': 'Thủ tướng'}]",[],False
2,Phạm Văn Đồng,Phạm Văn Đồng (1 tháng 3 năm 1906 - 29 tháng 4...,"Giai đoạn năm 1955-1976, Phạm Văn Đồng nắm giữ...",uit_000003,"[{'answer_start': 245, 'text': 'Thủ tướng Chín...",[],False
3,Phạm Văn Đồng,Phạm Văn Đồng (1 tháng 3 năm 1906 - 29 tháng 4...,Tên gọi nào được Phạm Văn Đồng sử dụng trước k...,uit_000004,[],"[{'answer_start': 507, 'text': 'Lâm Bá Kiệt'}]",True
4,Phạm Văn Đồng,Phạm Văn Đồng (1 tháng 3 năm 1906 - 29 tháng 4...,Hồ Học Lãm giữ chức vụ gì trong bộ máy Nhà nướ...,uit_000005,[],"[{'answer_start': 60, 'text': 'Thủ tướng'}]",True
...,...,...,...,...,...,...,...
26452,Chiến tranh Đông Dương,"Năm 1958, Cộng đồng Pháp khai sinh và Liên hiệ...",Thời kì Liên hiệp Pháp lãnh đạo kết thúc vào t...,uit_028453,"[{'answer_start': 0, 'text': 'Năm 1958'}]",[],False
26453,Chiến tranh Đông Dương,"Năm 1958, Cộng đồng Pháp khai sinh và Liên hiệ...",Các nước nào đã ký kết hiệp ước với Pháp để tu...,uit_028454,"[{'answer_start': 570, 'text': 'Algérie, Tunis...",[],False
26454,Chiến tranh Đông Dương,"Năm 1958, Cộng đồng Pháp khai sinh và Liên hiệ...",Quốc gia nào tiếp tục sử dụng biện pháp bạo lự...,uit_028455,"[{'answer_start': 288, 'text': 'Algérie'}]",[],False
26455,Chiến tranh Đông Dương,"Năm 1958, Cộng đồng Pháp khai sinh và Liên hiệ...",Điều 68 Hiến pháp Pháp (năm 1958) quy định nhữ...,uit_028456,[],"[{'answer_start': 114, 'text': 'mỗi quốc gia h...",True


In [49]:
val_df = pd.concat([negative_value[:1000], val_df]).sort_values('id').reset_index(drop=True)

In [50]:
test_df = pd.concat([negative_value[1000:], test_df]).sort_values('id').reset_index(drop=True)

In [51]:
v2_train_new = convert_to_json(train_df)
with open('./MRC_VLSP/v2_train_ViQuAD_new.json', 'w', encoding='utf8') as f:
    json.dump(v2_train_new, f, ensure_ascii=False, indent=4)

100%|██████████| 26457/26457 [00:03<00:00, 7454.54it/s]


In [52]:
to_df('/home/brownyeyes/WorkDirectory/Multidocs-MRC/MRC_VLSP/v2_train_ViQuAD_new.json')

,title,context,question,id,answer,plausible_answers,is_impossible
0,Phạm Văn Đồng,Phạm Văn Đồng (1 tháng 3 năm 1906 - 29 tháng 4...,Tên gọi nào được Phạm Văn Đồng sử dụng khi làm...,uit_000001,"[{'answer_start': 507, 'text': 'Lâm Bá Kiệt'}]",[],False
1,Phạm Văn Đồng,Phạm Văn Đồng (1 tháng 3 năm 1906 - 29 tháng 4...,Phạm Văn Đồng giữ chức vụ gì trong bộ máy Nhà ...,uit_000002,"[{'answer_start': 60, 'text': 'Thủ tướng'}]",[],False
2,Phạm Văn Đồng,Phạm Văn Đồng (1 tháng 3 năm 1906 - 29 tháng 4...,"Giai đoạn năm 1955-1976, Phạm Văn Đồng nắm giữ...",uit_000003,"[{'answer_start': 245, 'text': 'Thủ tướng Chín...",[],False
3,Phạm Văn Đồng,Phạm Văn Đồng (1 tháng 3 năm 1906 - 29 tháng 4...,Tên gọi nào được Phạm Văn Đồng sử dụng trước k...,uit_000004,[],"[{'answer_start': 507, 'text': 'Lâm Bá Kiệt'}]",True
4,Phạm Văn Đồng,Phạm Văn Đồng (1 tháng 3 năm 1906 - 29 tháng 4...,Hồ Học Lãm giữ chức vụ gì trong bộ máy Nhà nướ...,uit_000005,[],"[{'answer_start': 60, 'text': 'Thủ tướng'}]",True
...,...,...,...,...,...,...,...
26452,Chiến tranh Đông Dương,"Năm 1958, Cộng đồng Pháp khai sinh và Liên hiệ...",Thời kì Liên hiệp Pháp lãnh đạo kết thúc vào t...,uit_028453,"[{'answer_start': 0, 'text': 'Năm 1958'}]",[],False
26453,Chiến tranh Đông Dương,"Năm 1958, Cộng đồng Pháp khai sinh và Liên hiệ...",Các nước nào đã ký kết hiệp ước với Pháp để tu...,uit_028454,"[{'answer_start': 570, 'text': 'Algérie, Tunis...",[],False
26454,Chiến tranh Đông Dương,"Năm 1958, Cộng đồng Pháp khai sinh và Liên hiệ...",Quốc gia nào tiếp tục sử dụng biện pháp bạo lự...,uit_028455,"[{'answer_start': 288, 'text': 'Algérie'}]",[],False
26455,Chiến tranh Đông Dương,"Năm 1958, Cộng đồng Pháp khai sinh và Liên hiệ...",Điều 68 Hiến pháp Pháp (năm 1958) quy định nhữ...,uit_028456,[],"[{'answer_start': 114, 'text': 'mỗi quốc gia h...",True


In [53]:
v2_val_new = convert_to_json(val_df)
with open('./MRC_VLSP/v2_dev_ViQuAD.json', 'w', encoding='utf8') as f:
    json.dump(v2_val_new, f, ensure_ascii=False, indent=4)

100%|██████████| 3285/3285 [00:00<00:00, 8644.25it/s]


In [54]:
v2_test = convert_to_json(test_df)

100%|██████████| 3210/3210 [00:00<00:00, 8357.71it/s]


In [55]:
v2_test['version'] = '2.0'
with open('./MRC_VLSP/v2_test_ViQuAD.json', 'w', encoding='utf8') as f:
    json.dump(v2_test, f, ensure_ascii=False, indent=4)

In [56]:
to_df('./MRC_VLSP/v2_test_ViQuAD.json')

,title,context,question,id,answer,plausible_answers,is_impossible
0,Thực vật có hoa,"Thuật ngữ để chỉ thực vật hạt kín là ""Angiospe...",Robert Brown phát hiện sự tồn tại của lá mầm t...,uit_000047,[],"[{'answer_start': 855, 'text': 'năm 1827'}]",True
1,Thực vật có hoa,"Trong phần lớn các hệ thống phân loại, thực vậ...",Hệ thống nào xem thực vật có hoa ở cấp độ nhánh?,uit_000076,[],"[{'answer_start': 665, 'text': 'Hệ thống Dahlg...",True
2,Thực vật có hoa,Phân loại nội bộ của nhóm này đã qua nhiều tha...,Hệ thống phân loại nhóm thực vật Cronquist đượ...,uit_000084,[],"[{'answer_start': 179, 'text': 'năm 1968'}]",True
3,Thực vật có hoa,"Theo truyền thống, thực vật có hoa được chia t...",Trong hội nghị Cronquist chia thực vật có hoa ...,uit_000089,[],"[{'answer_start': 51, 'text': 'hai'}]",True
4,Thực vật có hoa,"Giống như ở thực vật hạt trần, các bào tử được...",Lá có thể quyết rũ được côn trùng và chim nhờ ...,uit_000125,[],"[{'answer_start': 1041, 'text': 'màu sắc và mù...",True
...,...,...,...,...,...,...,...
3205,Chiến tranh Đông Dương,Hoa Kỳ cần có quan hệ đồng minh với Pháp để th...,Nhật Bản đã làm gì để cảm ơn sự giải cứu các p...,uit_028362,[],"[{'answer_start': 430, 'text': 'cơ quan tình b...",True
3206,Chiến tranh Đông Dương,"Từ năm 1949, sau khi Nội chiến Trung Quốc kết ...","Ở Trung Quốc, cuộc chiến tranh nào diễn ra sau...",uit_028378,[],"[{'answer_start': 63, 'text': 'chiến tranh Tri...",True
3207,Chiến tranh Đông Dương,"Để đối phó với các áp lực chính trị, quân sự v...",Sự kiện gì diễn ra vào ngày 16/2/1949?,uit_028392,[],"[{'answer_start': 321, 'text': 'Bộ Ngoại giao ...",True
3208,Chiến tranh Đông Dương,"Đầu năm 1950, Hoa Kỳ bắt đầu gửi viện trợ quân...",Tại sao Pháp nhanh chóng viện trợ cho Mỹ ở Đôn...,uit_028397,[],"[{'answer_start': 79, 'text': 'Do chiến tranh ...",True
